# Portfolio Making
Prepared by: `Kuhgi Jotojot`

## 1. Library Prerequisites

In [21]:
# Essential Packages

import pandas as pd
import numpy as np
import os
import re
import math
import calendar
import random

In [3]:
path = "raw_data/"
closing_price_list_with_time = list()

for filename in os.listdir(path):
    file_path = os.path.join(path, filename)

    pattern = r"PSE_DLY_([A-Z]+),\s*"  # Updated regex to handle the comma and optional whitespace

    match = re.search(pattern, filename)
    if match:
        ticker = match.group(1)
    else:
        print(f"No ticker in file: {filename}")
        continue  # Skip this file

    # Read CSV and set 'time' as index
    prices = pd.read_csv(file_path, parse_dates=['time'])
    prices.set_index('time', inplace=True)

    # Keep only the 'close' column, rename it to the ticker
    prices = prices[['close']]
    prices = prices.rename(columns={'close': ticker})

    # Append to list
    closing_price_list_with_time.append(prices)

# Combine all into one DataFrame
closing_prices_with_time_df = pd.concat(closing_price_list_with_time, axis=1)

# Optional: Sort by date (index)
closing_prices_with_time_df = closing_prices_with_time_df[
    (closing_prices_with_time_df.index >= '2021-06-01') & 
    (closing_prices_with_time_df.index <= '2024-12-31')
]
closing_prices_with_time_df.sort_index(inplace=True)

## 2. Data Preprocessing

In [8]:
# Create an empty dataframe
returns = pd.DataFrame()

# Define the column Returns
returns['Mean Daily Return'] = closing_prices_with_time_df.mean()

returns['Annual Return'] = closing_prices_with_time_df.mean() * 252

# Define the column Volatility
returns['Standard Deviation'] = closing_prices_with_time_df.std()

### 2.1 Sharpe Ratio

Risk Free Rate is based on the PDEx Fixed Income (FI) 1Y Bloomberg Valuation (BVAL) (proxy) as of June 11, 2025


![BVAL](img/BVAL.png)




In [10]:
# Assume an annual risk-free rate (e.g., 5%)
annual_rf_rate = 0.05817
trading_days = 252
daily_rf_rate = annual_rf_rate / trading_days

In [11]:
returns['Sharpe Ratio'] = (returns['Mean Daily Return'] - daily_rf_rate) / returns['Standard Deviation']

### Dataset for Portfolio Selection

In [13]:
display(returns)

,Mean Daily Return,Annual Return,Standard Deviation,Sharpe Ratio
AC,676.870497,1.705714e+05,78.849235,8.584361
ACEN,6.577332,1.657488e+03,2.118033,3.105287
AEV,44.380576,1.118391e+04,6.771505,6.553985
AGI,10.494684,2.644660e+03,1.436235,7.306920
ALI,29.778680,7.504227e+03,3.757297,7.925498
AREIT,32.736982,8.249719e+03,3.843226,8.518039
BDO,116.766228,2.942509e+04,22.126345,5.277238
BLOOM,8.032190,2.024112e+03,1.893078,4.242805
BPI,95.633287,2.409959e+04,19.570207,4.886666
CBC,28.975869,7.301919e+03,9.989399,2.900639


## 3. Portfolio Making

In [ ]:
portfolio_metrics_df = returns.transpose()
portfolio_metrics_df

,AC,ACEN,AEV,AGI,ALI,AREIT,BDO,BLOOM,BPI,CBC,...,MER,MONDE,PGOLD,PSEI,SCC,SM,SMC,SMPH,TEL,URC
Mean Daily Return,676.870497,6.577332,44.380576,10.494684,29.778680,32.736982,116.766228,8.032190,95.633287,28.975869,...,308.549581,11.305741,28.898836,6.698053e+03,20.916361,876.430974,101.297599,32.250736,1258.481574,110.202364
Annual Return,170571.365236,1657.487598,11183.905096,2644.660288,7504.227356,8249.719412,29425.089578,2024.111803,24099.588260,7301.919108,...,77754.494333,2849.046830,7282.506634,1.687909e+06,5270.923079,220860.605390,25526.995070,8127.185474,317137.356705,27770.995714
Standard Deviation,78.849235,2.118033,6.771505,1.436235,3.757297,3.843226,22.126345,1.893078,19.570207,9.989399,...,63.115598,2.892488,3.859887,3.602520e+02,6.372390,59.651517,6.774066,2.929120,158.490411,13.502621
Sharpe Ratio,8.584361,3.105287,6.553985,7.306920,7.925498,8.518039,5.277238,4.242805,4.886666,2.900639,...,4.888639,3.908576,7.486904,1.859268e+01,3.282306,14.692514,14.953703,11.010306,7.940426,8.161536


In [23]:
clusters_dict = {
    0 : ['GLO', 'TEL'],
    1 : ['ACEN', 'SCC'],
    2 : ['AC', 'AEV', 'AGI', 'ALI', 'AREIT', 'CNVRG', 'JGS', 'LTG', 'PGOLD', 'SM', 'SMC', 'SMPH', 'URC'],
    3 : ['BLOOM', 'GTCAP', 'MONDE'],
    4 : ['BDO', 'BPI', 'CBC', 'CNPF', 'DMC', 'EMI', 'ICT', 'JFC', 'MBT', 'MER']
}

# Portfolio Making (Minimize Standard Deviation)

print('Portfolio 1: Get stock from each clusters with the minimum standard deviation')
for i in clusters_dict.keys():   
    temp_df = portfolio_metrics_df[clusters_dict[i]]
    std_row = temp_df.iloc[2]
    min_std = std_row.idxmin()
    print(f"{i}: {min_std}")

print("\n")

print('Portfolio 2: Get stock from each clusters with the maximum Sharpe Ratio')
for i in clusters_dict.keys():   
    temp_df = portfolio_metrics_df[clusters_dict[i]]
    shr_row = temp_df.iloc[3]
    max_shr = shr_row.idxmax()
    print(f"{i}: {max_shr}")

print("\n")

print('Portfolio 3: Get stock from each clusters with the maximum annual returns')
for i in clusters_dict.keys():   
    temp_df = portfolio_metrics_df[clusters_dict[i]]
    anr_row = temp_df.iloc[1]
    max_anr = anr_row.idxmax()
    print(f"{i}: {max_anr}")

print("\n")

random.seed(14)

print('Portfolio 4: Get random stock from each clusters')
for i in clusters_dict.keys():   
    random_stock = random.choice(clusters_dict[i])
    print(f"{i}: {random_stock}")


Portfolio 1: Get stock from each clusters with the minimum standard deviation
0: TEL
1: ACEN
2: LTG
3: BLOOM
4: DMC


Portfolio 2: Get stock from each clusters with the maximum Sharpe Ratio
0: TEL
1: SCC
2: SMC
3: GTCAP
4: JFC


Portfolio 3: Get stock from each clusters with the maximum annual returns
0: GLO
1: SCC
2: SM
3: GTCAP
4: MER


Portfolio 4: Get random stock from each clusters
0: GLO
1: ACEN
2: AREIT
3: MONDE
4: DMC
